In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import lightgbm as lgb
from sklearn import preprocessing, metrics
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
import gc
from tqdm import tqdm
from scipy.sparse import csr_matrix
from  datetime import datetime, timedelta

# Random Forestの回帰と分類
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [2]:
cd C:\Users\minih\python_prac1\m5\git-repository\kaggle-m5-accuracy-library\library

C:\Users\minih\python_prac1\m5\git-repository\kaggle-m5-accuracy-library\library


In [3]:
from patair import reduce_mem_usage

In [4]:
cd C:\Users\minih\python_prac1\library\m5

C:\Users\minih\python_prac1\library\m5


In [5]:
product = pd.read_pickle("product.pickle").pipe(reduce_mem_usage)

Mem. usage decreased to  0.64 Mb (0.0% reduction)


In [6]:
datas = pd.read_pickle("all_sales_train_validation_translated_contain_null.pickle")

In [7]:
check_feature=[
               'prev_28', 'p_prev_1','rolling_mean_t7', 
               'rolling_mean_t14', 'price_change_t1','rolling_std_t7', 'rolling_std_t14',
               'rolling_price_std_t7', 'rolling_price_std_t30', 'price_change_t1', 'rolling_price_std_t7',
               ]

In [8]:
datas.dropna(subset=check_feature, inplace=True)

In [9]:
#WRMSSEの計算用(kernelより)

NUM_ITEMS = 30490
DAYS_PRED = 28

weight_mat = np.c_[np.ones([NUM_ITEMS,1]).astype(np.int8), # level 1
                   pd.get_dummies(product.state_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.store_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.cat_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.dept_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.state_id.astype(str) + product.cat_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.state_id.astype(str) + product.dept_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.store_id.astype(str) + product.cat_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.store_id.astype(str) + product.dept_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.item_id.astype(str),drop_first=False).astype('int8').values,
                   pd.get_dummies(product.state_id.astype(str) + product.item_id.astype(str),drop_first=False).astype('int8').values,
                   np.identity(NUM_ITEMS).astype(np.int8) #item :level 12
                   ].T

weight_mat_csr = csr_matrix(weight_mat)
del weight_mat; gc.collect()

def weight_calc(data,product):
    
    # calculate the denominator of RMSSE, and calculate the weight base on sales amount

    sales_train_val = pd.read_csv('sales_train_validation.csv')

    d_name = ['d_' + str(i+1) for i in range(1913)]

    sales_train_val = weight_mat_csr * sales_train_val[d_name].values

    # calculate the start position(first non-zero demand observed date) for each item / 商品の最初の売上日
    # 1-1914のdayの数列のうち, 売上が存在しない日を一旦0にし、0を9999に置換。そのうえでminimum numberを計算
    df_tmp = ((sales_train_val>0) * np.tile(np.arange(1,1914),(weight_mat_csr.shape[0],1)))

    start_no = np.min(np.where(df_tmp==0,9999,df_tmp),axis=1)-1

    flag = np.dot(np.diag(1/(start_no+1)) , np.tile(np.arange(1,1914),(weight_mat_csr.shape[0],1)))<1

    sales_train_val = np.where(flag,np.nan,sales_train_val)

    # denominator of RMSSE / RMSSEの分母
    weight1 = np.nansum(np.diff(sales_train_val,axis=1)**2,axis=1)/(1913-start_no)

    # calculate the sales amount for each item/level
    df_tmp = data[(data['date'] > '2016-03-27') & (data['date'] <= '2016-04-24')]
    df_tmp['sell_num'] = df_tmp['sell_num'] * df_tmp['sell_price']
    df_tmp =df_tmp.groupby(['id'])['sell_num'].apply(np.sum)
    df_tmp = df_tmp[product.id].values
    
    weight2 = weight_mat_csr * df_tmp 

    weight2 = weight2/np.sum(weight2)

    del sales_train_val
    gc.collect()
    
    return weight1, weight2

#weight1, weight2 = weight_calc(datas,product)

def wrmsse(preds, data):
    
    # this function is calculate for last 28 days to consider the non-zero demand period
    
    # actual obserbed values / 正解ラベル
    y_true = data.get_label()
    
    y_true = y_true[-(NUM_ITEMS * DAYS_PRED):]
    preds = preds[-(NUM_ITEMS * DAYS_PRED):]
    # number of columns
    num_col = DAYS_PRED
    
    # reshape data to original array((NUM_ITEMS*num_col,1)->(NUM_ITEMS, num_col) ) / 推論の結果が 1 次元の配列になっているので直す
    #print(preds)
    #print(y_true)
    reshaped_preds = preds.reshape(num_col, NUM_ITEMS).T
    reshaped_true = np.array(y_true).reshape(num_col, NUM_ITEMS).T
    
          
    train = weight_mat_csr*np.c_[reshaped_preds, reshaped_true]
    #print(train)
    
    score = np.sum(
                np.sqrt(
                    np.mean(
                        np.square(
                            train[:,:num_col] - train[:,num_col:])
                        ,axis=1) / weight1) * weight2)
    
    return 'wrmsse', score, False

def wrmsse_simple(preds, data):
    
    # actual obserbed values / 正解ラベル
    y_true = data.get_label()
    
    y_true = y_true[-(NUM_ITEMS * DAYS_PRED):]
    preds = preds[-(NUM_ITEMS * DAYS_PRED):]
    # number of columns
    num_col = DAYS_PRED
    
    # reshape data to original array((NUM_ITEMS*num_col,1)->(NUM_ITEMS, num_col) ) / 推論の結果が 1 次元の配列になっているので直す
    reshaped_preds = preds.reshape(num_col, NUM_ITEMS).T
    reshaped_true = y_true.reshape(num_col, NUM_ITEMS).T
          
    train = np.c_[reshaped_preds, reshaped_true]
   
    
    weight2_2 = weight2[:NUM_ITEMS]
    weight2_2 = weight2_2/np.sum(weight2_2)
    
    score = np.sum(
                np.sqrt(
                    np.mean(
                        np.square(
                            train[:,:num_col] - train[:,num_col:])
                        ,axis=1) /  weight1[:NUM_ITEMS])*weight2_2)
    
    return 'wrmsse', score, False

In [10]:
#weight_mat_csr
weights = pd.read_pickle("weight.pickle")
weight1 = weights[weights.index ==0]
weight2 = weights[weights.index ==1]
weight1=weight1.values
weight2=weight2.values

In [11]:
#x_train = datas[datas['date'] < '2016-04-18']
x_train = datas[datas['date'] < '2016-03-28']
y_train = x_train['sell_num']
#x_val = datas[datas['date'] >= '2016-04-18']
x_val = datas[(datas['date'] >= '2016-03-28')& (datas['date'] <= '2016-04-24')]
y_val = x_val['sell_num']
test = datas[datas['date'] > '2016-04-24']
del datas
gc.collect()

40

In [12]:
features = [
   #訓練に用いる特徴量をここに記述
    "item_id",
    "dept_id",
    "cat_id",
    "store_id",
    "state_id",
    "event_name_1",
    #"event_type_1",
    "event_name_2",
    #"event_type_2",
    
    
    "snap_CA",
    "snap_TX",
    "snap_WI",
    
    "sell_price",
    # demand features.
    #'prev_1',
    #'prev_2',
    #'prev_7', 
    #'prev_14', 
    'prev_28',
    #'prev_182', 
    #'prev_365', 
    'p_prev_1', 
    'rolling_mean_t7',
    'rolling_mean_t14', 
    #'rolling_mean_t30',
    'rolling_std_t7',
    'rolling_std_t14',
    #'rolling_std_t30',
    #'rolling_mean_t60', 
    #'rolling_mean_t90',
    #'rolling_mean_t180',  
    # price features
    "price_change_t1",
    #"price_change_t365",
    #'rolling_price_max_t365', 
    #'price_change_t365',
    "rolling_price_std_t7",
    "rolling_price_std_t30",
    # time features.
    "year",
    "month",
    "week",
    "day",
    "dayofweek",
    #"is_year_end",
    #"is_year_start",
    #"is_quarter_end",
    #"is_quarter_start",
    #"is_month_end",
    #"is_month_start",
    #"is_weekend",
]

In [13]:
#kernelから取ってきたパラメータ
params = {
        "objective" : "poisson",
        "metric" : "custom",
        #"metric" : "rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
#         "nthread" : 4
        'verbosity': 1,
        'num_iterations' : 1200,
        'num_leaves': 128,
        "min_data_in_leaf": 100,
        "random_state": 32,
}

In [14]:
rfr = RandomForestRegressor()

In [18]:
rfr.fit(x_train[features],y_train)

MemoryError: could not allocate 939524096 bytes

In [ ]:
val_pred = rfr.predict(x_val[features])

In [ ]:
buff, score, buff2 = wrmsse(np.array(val_pred), val_set)

In [ ]:
score

In [ ]:
y_pred = rfr.predict(test[features])